In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
! git clone https://github.com/KaiyangZhou/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 9809 (delta 188), reused 267 (delta 183), pack-reused 9526
Receiving objects: 100% (9809/9809), 9.59 MiB | 1.54 MiB/s, done.
Resolving deltas: 100% (7226/7226), done.


In [ ]:
!ls


deep-person-reid  gdrive  sample_data


In [4]:
cd deep-person-reid/

/content/deep-person-reid


In [5]:
! pip install -r requirements.txt


     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 185 kB 44.7 MB/s 
     |████████████████████████████████| 42 kB 937 kB/s 
     |████████████████████████████████| 42 kB 840 kB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.1
    Uninstalling importlib-metadata-4.8.1:
      Successfully uninstalled importlib-metadata-4.8.1


In [6]:
!pip install torch torchvision torchaudio

     |████████████████████████████████| 2.9 MB 4.3 MB/s 
     |████████████████████████████████| 1.9 MB 32.5 MB/s 
     |████████████████████████████████| 1.9 MB 31.8 MB/s 


In [7]:
!python setup.py develop

/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/deep-person-reid/torchreid/metrics/rank_cylib/rank_cy.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running develop
running egg_info
creating torchreid.egg-info
writing torchreid.egg-info/PKG-INFO
writing dependency_links to torchreid.egg-info/dependency_links.txt
writing requirements to torchreid.egg-info/requires.txt
writing top-level names to torchreid.egg-info/top_level.txt
writing manifest file 'torchreid.egg-info/SOURCES.txt'

In [8]:
import torchreid

In [9]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3350 KB/s, 44 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [10]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [11]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [12]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=40,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: [1/40][10/404]	time 0.452 (1.022)	data 0.000 (0.547)	eta 4:35:04	loss 6.7177 (6.7454)	acc 3.1250 (0.6250)	lr 0.000300
epoch: [1/40][20/404]	time 0.452 (0.736)	data 0.001 (0.274)	eta 3:17:57	loss 7.0158 (6.9274)	acc 0.0000 (0.7812)	lr 0.000300
epoch: [1/40][30/404]	time 0.450 (0.641)	data 0.000 (0.183)	eta 2:52:14	loss 6.8925 (6.8500)	acc 0.0000 (1.1458)	lr 0.000300
epoch: [1/40][40/404]	time 0.449 (0.593)	data 0.001 (0.137)	eta 2:39:19	loss 6.6617 (6.7947)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/40][50/404]	time 0.449 (0.565)	data 0.001 (0.110)	eta 2:31:35	loss 6.4665 (6.7531)	acc 0.0000 (0.8750)	lr 0.000300
epoch: [1/40][60/404]	time 0.447 (0.545)	data 0.000 (0.092)	eta 2:26:20	loss 6.8422 (6.7334)	acc 0.0000 (0.8333)	lr 0.000300
epoch: [1/40][70/404]	time 0.450 (0.532)	data 0.004 (0.079)	eta 2:22:37	loss 6.4651 (6.7059)	acc 0.0000 (0.8036)	lr 0.000300
epoch: [1/40][80/404]	time 0.470 (0.522)	data 0.007 (0.069)	eta 2:19:48	loss 6.5336 (6.6896)	acc 0.0000 (0.8203)	lr 0.000300
